# Vexa Test Library Demo

This notebook demonstrates how to use the Vexa Test Library for comprehensive testing scenarios.

## Features:
- Create multiple users with individual API keys
- Random mapping of users to meetings
- Bot lifecycle management
- Background monitoring with timestamps
- Pandas integration for data analysis


In [ ]:
# Import required libraries
import os
import sys
import pandas as pd
from IPython.display import clear_output, display
import time

# Add the new_tests directory to the path
sys.path.append('./new_tests')

# Import our test library
from test_suite import TestSuite
from bot import Bot


In [ ]:
# Configuration
BASE_URL = "http://localhost:18056"  # Change to your Vexa API URL
ADMIN_API_KEY = 'token'  # Set this environment variable

# Test parameters
NUM_USERS = 12
MEETING_URLS = [
    "https://meet.google.com/grw-hxrw-dtf",
    "https://meet.google.com/nbk-dfdu-xkj"
]



In [ ]:
# Initialize TestSuite
test_suite = TestSuite(
    base_url=BASE_URL,
    admin_api_key=ADMIN_API_KEY,
    poll_interval=2.0 
)

print("TestSuite initialized")


In [ ]:
# Step 1: Create users
test_suite.cleanup()
users = test_suite.create_users(NUM_USERS)
print(f"Created {len(users)} users")
# Step 2: Create random user-meeting mapping
mapping = test_suite.create_random_mapping(MEETING_URLS)
bots = test_suite.create_bots(bot_name_prefix="DemoBot")
results = test_suite.start_all_bots(language='en', task='transcribe')
test_suite.start_monitoring()

In [ ]:
pd.options.display.max_columns = None
now = pd.Timestamp.now()

df = test_suite.get_latest_dataframe()
print(df['transcription_latency'].mean())
df[['bot_id','platform','meeting_status','created_at','transcription_latency','current_status','detected_languages','segments_count']].sort_values(by='transcription_latency', ascending=False)

In [ ]:
bot = bots[5]
bot.user_client.get_meetings()

In [ ]:
[s['text'] for s in bot.get_transcript()['segments']]

In [ ]:
bot.get_transcript()

In [ ]:
test_suite.cleanup()